In [2]:
import boto3
import sagemaker
import pandas as pd
import seaborn as sns
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect
import matplotlib.pyplot as plt

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
s3_client = boto3.client('s3', region_name='us-east-1')
bucket_name = 'team1ads508finalproject'

In [4]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

s3_public = "s3://team1ads508finalproject"

s3_private = "s3://{}/ADS508_project".format(bucket)

!aws s3 cp --recursive $s3_public/ $s3_private/

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
copy: s3://team1ads508finalproject/application_train.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/application_train.csv
copy: s3://team1ads508finalproject/previous_application.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/previous_application.csv
copy: s3://team1ads508finalproject/bureau.csv to s3://sagemaker-us-east-1-654654578488/ADS508_project/bureau.csv


In [5]:
print(s3_private)

s3://sagemaker-us-east-1-654654578488/ADS508_project


In [6]:
%store

Stored variables and their in-db values:


In [7]:
database_name = "dsoaws"

In [8]:
#Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [9]:
conn = connect(region_name='us-east-1', s3_staging_dir=s3_staging_dir)

In [10]:
# SQL queries for each of the csv's
sql_queries = {
    'bureau': 'SELECT * FROM bureau LIMIT 30000',
    'previous_application': 'SELECT * FROM previous_application LIMIT 30000',
    'application': 'SELECT * FROM application LIMIT 30000'
}

# Fetch data and convert to DataFrames
bureau_df = pd.read_sql(sql_queries['bureau'], conn)
prev_app_df = pd.read_sql(sql_queries['previous_application'], conn)
app_df = pd.read_sql(sql_queries['application'], conn)

# Display each pandas Dataframe
print("bureau_df:")
print(bureau_df.head())

print("\nprev_app_df:")
print(prev_app_df.head())

print("\napp_df:")
print(app_df.head())

/tmp/ipykernel_5296/3237024562.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bureau_df = pd.read_sql(sql_queries['bureau'], conn)
/tmp/ipykernel_5296/3237024562.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prev_app_df = pd.read_sql(sql_queries['previous_application'], conn)
/tmp/ipykernel_5296/3237024562.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  app_df = pd.read_sql(sql_queries['application'], conn)


bureau_df:
   sk_id_curr  sk_id_bureau credit_active credit_currency  days_credit  \
0    451700.0     5153425.0        Closed      currency 1      -1401.0   
1    451700.0     5153427.0        Closed      currency 1      -1926.0   
2    451700.0     5153428.0        Closed      currency 1      -1214.0   
3    451700.0     5153429.0        Closed      currency 1       -716.0   
4    451700.0     5153430.0        Closed      currency 1       -469.0   

   credit_day_overdue  days_credit_enddate  days_enddate_fact  \
0                 0.0              -1220.0            -1240.0   
1                 0.0              -1561.0            -1561.0   
2                 0.0               -908.0             -908.0   
3                 0.0               -410.0             -410.0   
4                 0.0               -165.0             -193.0   

   amt_credit_max_overdue  cnt_credit_prolong  amt_credit_sum  \
0                     NaN                 0.0         26928.0   
1                     N

In [11]:
# bureau_df check for duplicates
bureau_df = bureau_df.drop_duplicates()
print("Shape after removing duplicate:", bureau_df.shape)

Shape after removing duplicate: (30000, 17)


In [12]:
# prev_app_df check for duplicates
prev_app_df = prev_app_df.drop_duplicates()
print("Shape after removing duplicates:", prev_app_df.shape)

Shape after removing duplicates: (30000, 37)


In [13]:
#app_df check for duplicates
print("Before removing duplicates - app_df shape:", app_df.shape)
app_df = app_df.drop_duplicates()
print("Shape after removing duplicate:", app_df.shape)

Before removing duplicates - app_df shape: (30000, 122)
Shape after removing duplicate: (30000, 122)


In [14]:
# Aggregate data for bureau_df
bureau_df_grouped = bureau_df.groupby('sk_id_curr').agg('first').reset_index()

# Aggregate data for prev_app_df
prev_app_df_grouped = prev_app_df.groupby('sk_id_curr').agg('first').reset_index()

# Aggregate data for app_df
app_df_grouped = app_df.groupby('sk_id_curr').agg('first').reset_index()

# Concatenate the grouped DataFrames
aggregated_df = pd.concat([bureau_df_grouped, prev_app_df_grouped, app_df_grouped], ignore_index=True)

# Display sample data from the aggregated DataFrame
print(aggregated_df.head())

   sk_id_curr  sk_id_bureau credit_active credit_currency  days_credit  \
0    100039.0     5153442.0        Active      currency 1        -91.0   
1    100053.0     5723577.0        Closed      currency 1      -1764.0   
2    100179.0     6731344.0        Closed      currency 1      -1704.0   
3    100229.0     5309712.0        Closed      currency 1       -401.0   
4    100270.0     6728606.0        Closed      currency 1       -742.0   

   credit_day_overdue  days_credit_enddate  days_enddate_fact  \
0                 0.0               1379.0            -1406.0   
1                 0.0               -648.0             -648.0   
2                 0.0              -1338.0            -1332.0   
3                 0.0               -279.0             -278.0   
4                 0.0               -591.0             -620.0   

   amt_credit_max_overdue  cnt_credit_prolong  ...  flag_document_18  \
0                     NaN                 0.0  ...               NaN   
1                   

In [15]:
duplicates = aggregated_df.duplicated(subset=['sk_id_curr'])
print("Duplicate rows:", duplicates.sum())


Duplicate rows: 2598


In [16]:
final_df = aggregated_df.drop_duplicates()
duplicates = final_df.duplicated(subset=['sk_id_curr'])
print("Duplicate rows:", duplicates.sum())

Duplicate rows: 2598


In [17]:
print(aggregated_df.columns)

Index(['sk_id_curr', 'sk_id_bureau', 'credit_active', 'credit_currency',
       'days_credit', 'credit_day_overdue', 'days_credit_enddate',
       'days_enddate_fact', 'amt_credit_max_overdue', 'cnt_credit_prolong',
       ...
       'flag_document_18', 'flag_document_19', 'flag_document_20',
       'flag_document_21', 'amt_req_credit_bureau_hour',
       'amt_req_credit_bureau_day', 'amt_req_credit_bureau_week',
       'amt_req_credit_bureau_mon', 'amt_req_credit_bureau_qrt',
       'amt_req_credit_bureau_year'],
      dtype='object', length=166)


In [18]:
final_df = aggregated_df[['sk_id_curr', 'target', 'name_contract_type', 'code_gender',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'amt_credit', 'amt_annuity', 'credit_active', 'amt_income_total',
       'amt_credit', 'amt_annuity', 'name_type_suite', 'name_income_type',
       'name_education_type', 'name_family_status', 'name_housing_type', 'weekday_appr_process_start', 'organization_type', 'name_cash_loan_purpose', 'name_contract_status', 'days_decision',
       'name_payment_type', 'code_reject_reason', 'name_type_suite', 'name_client_type', 'name_portfolio']]
final_df

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,...,weekday_appr_process_start,organization_type,name_cash_loan_purpose,name_contract_status,days_decision,name_payment_type,code_reject_reason,name_type_suite,name_client_type,name_portfolio
0,100039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100053.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100179.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55835,2844983.0,278499.0,Cash loans,,0.0,0.0,NaN,NaN,NaN,11.0,...,,None,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
55836,2845133.0,368565.0,Cash loans,,0.0,0.0,NaN,NaN,NaN,14.0,...,,None,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
55837,2845296.0,310414.0,Cash loans,,0.0,0.0,NaN,NaN,NaN,11.0,...,,None,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
55838,2845317.0,147424.0,Cash loans,,0.0,0.0,NaN,NaN,NaN,11.0,...,,None,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [19]:
# Unique Values
unique = aggregated_df.nunique()
unique

sk_id_curr                    53242
sk_id_bureau                   6272
credit_active                     4
credit_currency                   2
days_credit                    2474
                              ...  
amt_req_credit_bureau_day         7
amt_req_credit_bureau_week        5
amt_req_credit_bureau_mon        16
amt_req_credit_bureau_qrt         7
amt_req_credit_bureau_year       13
Length: 166, dtype: int64

In [20]:
# Null Values
null_values = aggregated_df.isnull().sum()
null_values

sk_id_curr                        0
sk_id_bureau                  49568
credit_active                 49568
credit_currency               49568
days_credit                   49568
                              ...  
amt_req_credit_bureau_day     50947
amt_req_credit_bureau_week    50950
amt_req_credit_bureau_mon     50950
amt_req_credit_bureau_qrt     50950
amt_req_credit_bureau_year    50950
Length: 166, dtype: int64

In [21]:
# Drop columns with null values
final_df = aggregated_df.dropna()

# Display Clean Dataframe
print(final_df)


Empty DataFrame
Columns: [sk_id_curr, sk_id_bureau, credit_active, credit_currency, days_credit, credit_day_overdue, days_credit_enddate, days_enddate_fact, amt_credit_max_overdue, cnt_credit_prolong, amt_credit_sum, amt_credit_sum_debt, amt_credit_sum_limit, amt_credit_sum_overdue, credit_type, days_credit_update, amt_annuity, sk_id_prev, name_contract_type, amt_application, amt_credit, amt_down_payment, amt_goods_price, weekday_appr_process_start, hour_appr_process_start, flag_last_appl_per_contract, nflag_last_appl_in_day, rate_down_payment, rate_interest_primary, rate_interest_privileged, name_cash_loan_purpose, name_contract_status, days_decision, name_payment_type, code_reject_reason, name_type_suite, name_client_type, name_goods_category, name_portfolio, name_product_type, channel_type, sellerplace_area, name_seller_industry, cnt_payment, name_yield_group, product_combination, days_first_drawing, days_first_due, days_last_due_1st_version, days_last_due, days_termination, nflag_i

In [22]:
final_df.describe()

,sk_id_curr,sk_id_bureau,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,...,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
print(final_df.columns)

Index(['sk_id_curr', 'sk_id_bureau', 'credit_active', 'credit_currency',
       'days_credit', 'credit_day_overdue', 'days_credit_enddate',
       'days_enddate_fact', 'amt_credit_max_overdue', 'cnt_credit_prolong',
       ...
       'flag_document_18', 'flag_document_19', 'flag_document_20',
       'flag_document_21', 'amt_req_credit_bureau_hour',
       'amt_req_credit_bureau_day', 'amt_req_credit_bureau_week',
       'amt_req_credit_bureau_mon', 'amt_req_credit_bureau_qrt',
       'amt_req_credit_bureau_year'],
      dtype='object', length=166)


In [24]:
# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(final_df, annot=True, cmap='viridis', fmt='d')
plt.title('Feature Assessment Heatmap')
plt.show()

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1200x800 with 0 Axes>